In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, ridge_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

In [2]:
df = pd.read_csv('data/final_dataset.csv')
print(df.shape)
df.head()

(1190, 54)


,NAME,ISO_TIME,LAT,LON,filename,year_sine,day_sine,moon_phase,vel_ir_x_1,vel_ir_y_1,...,sheer_ir_5,sheer_ir_6,sheer_ir_7,sheer_wv_1,sheer_wv_2,sheer_wv_3,sheer_wv_4,sheer_wv_5,sheer_wv_6,sheer_wv_7
0,GONU,31-05-2007 12:00,14.0,71.7,20070531120000,0.530730,-3.216245e-16,0.107425,-5.466686,-12.396900,...,-0.910754,-3.226571,0.809995,0.950572,-0.506926,9.826063,2.595220,-2.041054,-2.620516,0.061143
1,GONU,31-05-2007 15:00,14.1,71.7,20070531150000,0.530730,-7.071068e-01,0.082893,-0.400851,9.252213,...,0.143213,-2.477745,-0.071372,0.387157,-0.400879,-1.115540,4.246338,2.632915,0.405643,-0.522360
2,GONU,31-05-2007 18:00,14.2,71.5,20070531180000,0.530730,-1.000000e+00,0.058265,-6.725895,-10.947097,...,0.591151,0.799708,0.812780,-0.615410,12.071303,-1.607280,-3.550708,0.997142,0.718704,1.283197
3,GONU,31-05-2007 21:00,14.3,71.3,20070531210000,0.530730,-7.071068e-01,0.033555,-7.249054,-14.104602,...,-0.099546,-0.145392,-10.752103,-0.820789,-0.392546,-0.100164,0.467178,-0.449377,-1.045755,2.532311
4,GONU,01-06-2007 00:00,14.4,70.9,20070601000000,0.516062,0.000000e+00,0.008777,-0.261981,-6.099148,...,1.306643,1.169234,10.600911,0.021447,-0.378746,0.754751,14.445832,-0.024937,-0.472799,-3.566408


In [3]:
df.NAME.unique(), len(df.NAME.unique())

(array(['GONU', 'SIDR', 'NARGIS', 'BIJLI', 'AILA', 'PHYAN', 'WARD',
        'LAILA', 'GIRI', 'JAL', 'KEILA', 'THANE', 'MURJAN', 'NILAM',
        'HELEN', 'NANAUK', 'HUDHUD', 'NILOFAR', 'ASHOBAA', 'KOMEN',
        'CHAPALA', 'MEGH', 'ROANU', 'KYANT', 'NADA', 'VARDAH'],
       dtype=object),
 26)

In [4]:
# 0 : ARB, 1 : BOB 
basin_map = {
    'GONU': 0, 'SIDR': 1, 'NARGIS': 1, 'BIJLI': 1, 'AILA': 1, 'PHYAN': 0,
    'WARD': 1, 'LAILA': 1, 'GIRI': 1, 'JAL': 1, 'KEILA': 0, 'THANE': 1,
    'MURJAN': 0, 'NILAM': 1, 'HELEN': 1, 'NANAUK': 0, 'HUDHUD': 1,
    'NILOFAR': 0, 'ASHOBAA': 0, 'KOMEN': 1, 'CHAPALA': 0, 'MEGH': 0,
    'ROANU': 1, 'KYANT': 1, 'NADA': 1, 'VARDAH': 1
}

df['BASIN'] = df['NAME'].map(basin_map)

In [5]:
df.columns, df.shape

(Index(['NAME', 'ISO_TIME', 'LAT', 'LON', 'filename', 'year_sine', 'day_sine',
        'moon_phase', 'vel_ir_x_1', 'vel_ir_y_1', 'vel_ir_x_2', 'vel_ir_y_2',
        'vel_ir_x_3', 'vel_ir_y_3', 'vel_ir_x_4', 'vel_ir_y_4', 'vel_ir_x_5',
        'vel_ir_y_5', 'vel_ir_x_6', 'vel_ir_y_6', 'vel_ir_x_7', 'vel_ir_y_7',
        'vel_ir_x_8', 'vel_ir_y_8', 'vel_wv_x_1', 'vel_wv_y_1', 'vel_wv_x_2',
        'vel_wv_y_2', 'vel_wv_x_3', 'vel_wv_y_3', 'vel_wv_x_4', 'vel_wv_y_4',
        'vel_wv_x_5', 'vel_wv_y_5', 'vel_wv_x_6', 'vel_wv_y_6', 'vel_wv_x_7',
        'vel_wv_y_7', 'vel_wv_x_8', 'vel_wv_y_8', 'sheer_ir_1', 'sheer_ir_2',
        'sheer_ir_3', 'sheer_ir_4', 'sheer_ir_5', 'sheer_ir_6', 'sheer_ir_7',
        'sheer_wv_1', 'sheer_wv_2', 'sheer_wv_3', 'sheer_wv_4', 'sheer_wv_5',
        'sheer_wv_6', 'sheer_wv_7', 'BASIN'],
       dtype='object'),
 (1190, 55))

$\frac{dx}{dt} = \alpha F(U_{wv}) + (1-\alpha) G(U_{ir}) - W_{\beta} - W_{\beta} e^\gamma \exp(H(\frac{dU_{wv}}{dV_{wv}}, \frac{dU_{ir}}{dV_{ir}}))$ 

$\frac{dy}{dt} = \alpha F(U_{wv}) + (1-\alpha) G(U_{ir}) + W_{\beta} + W_{\beta} e^\gamma \exp(H(\frac{dU_{wv}}{dV_{wv}}, \frac{dU_{ir}}{dV_{ir}}))$ 

In [6]:
dx = None
dy = None
flag = False

for cyclone in df.NAME.unique():
    lon = df[df.NAME == cyclone].LON.diff()
    lat = df[df.NAME == cyclone].LAT.diff()

    if not flag and not dx:
        dy = lat
        dx = lon
        flag = True

    else:
        dy = pd.concat([dy, lat], ignore_index=True)
        dx = pd.concat([dx, lon], ignore_index=True)

df['dx'] = dx
df['dy'] = dy

df.head()

,NAME,ISO_TIME,LAT,LON,filename,year_sine,day_sine,moon_phase,vel_ir_x_1,vel_ir_y_1,...,sheer_wv_1,sheer_wv_2,sheer_wv_3,sheer_wv_4,sheer_wv_5,sheer_wv_6,sheer_wv_7,BASIN,dx,dy
0,GONU,31-05-2007 12:00,14.0,71.7,20070531120000,0.530730,-3.216245e-16,0.107425,-5.466686,-12.396900,...,0.950572,-0.506926,9.826063,2.595220,-2.041054,-2.620516,0.061143,0,NaN,NaN
1,GONU,31-05-2007 15:00,14.1,71.7,20070531150000,0.530730,-7.071068e-01,0.082893,-0.400851,9.252213,...,0.387157,-0.400879,-1.115540,4.246338,2.632915,0.405643,-0.522360,0,0.0,0.1
2,GONU,31-05-2007 18:00,14.2,71.5,20070531180000,0.530730,-1.000000e+00,0.058265,-6.725895,-10.947097,...,-0.615410,12.071303,-1.607280,-3.550708,0.997142,0.718704,1.283197,0,-0.2,0.1
3,GONU,31-05-2007 21:00,14.3,71.3,20070531210000,0.530730,-7.071068e-01,0.033555,-7.249054,-14.104602,...,-0.820789,-0.392546,-0.100164,0.467178,-0.449377,-1.045755,2.532311,0,-0.2,0.1
4,GONU,01-06-2007 00:00,14.4,70.9,20070601000000,0.516062,0.000000e+00,0.008777,-0.261981,-6.099148,...,0.021447,-0.378746,0.754751,14.445832,-0.024937,-0.472799,-3.566408,0,-0.4,0.1


In [7]:
# df = df[df.BASIN == 1]
df = df[df.columns.drop(list(df.filter(regex='bt_')) + ['ISO_TIME', 'filename', 'year_sine', 'day_sine', 'moon_phase', 'BASIN'])]

df.dropna(inplace=True)
df = df[df.columns.drop(['NAME', 'LAT', 'LON'])]

df.columns, df.shape

(Index(['vel_ir_x_1', 'vel_ir_y_1', 'vel_ir_x_2', 'vel_ir_y_2', 'vel_ir_x_3',
        'vel_ir_y_3', 'vel_ir_x_4', 'vel_ir_y_4', 'vel_ir_x_5', 'vel_ir_y_5',
        'vel_ir_x_6', 'vel_ir_y_6', 'vel_ir_x_7', 'vel_ir_y_7', 'vel_ir_x_8',
        'vel_ir_y_8', 'vel_wv_x_1', 'vel_wv_y_1', 'vel_wv_x_2', 'vel_wv_y_2',
        'vel_wv_x_3', 'vel_wv_y_3', 'vel_wv_x_4', 'vel_wv_y_4', 'vel_wv_x_5',
        'vel_wv_y_5', 'vel_wv_x_6', 'vel_wv_y_6', 'vel_wv_x_7', 'vel_wv_y_7',
        'vel_wv_x_8', 'vel_wv_y_8', 'sheer_ir_1', 'sheer_ir_2', 'sheer_ir_3',
        'sheer_ir_4', 'sheer_ir_5', 'sheer_ir_6', 'sheer_ir_7', 'sheer_wv_1',
        'sheer_wv_2', 'sheer_wv_3', 'sheer_wv_4', 'sheer_wv_5', 'sheer_wv_6',
        'sheer_wv_7', 'dx', 'dy'],
       dtype='object'),
 (1164, 48))

# Solver

In [8]:
X = df[df.columns.drop(['dy', 'dx'])].to_numpy()
X_hat = df[['dx', 'dy']].to_numpy()
y_dx = df['dx'].values
y_dy = df['dy'].values
X.shape, X_hat.shape

((1164, 46), (1164, 2))

In [9]:
# Brute force, all combinations. No intentionality.

def create_basic_library(u: np.ndarray, polynomial_order: int) -> np.ndarray:
    (m, n) = u.shape
    theta = np.ones((m, 1))

    # Polynomials of order 1.
    theta = np.hstack((theta, u))

    # Polynomials of order 2.
    if polynomial_order >= 2:
        for i in range(n):
            for j in range(i, n):
                theta = np.hstack((theta, u[:, i:i + 1] * u[:, j:j + 1]))

    # Polynomials of order 3.
    if polynomial_order >= 3:
        for i in range(n):
            for j in range(i, n):
                for k in range(j, n):
                    theta = np.hstack(
                        (theta, u[:, i:i + 1] * u[:, j:j + 1] * u[:, k:k + 1]))

    return theta

In [10]:
# Chunked library to accomodate my differential equations

def create_chunked_library(u: np.ndarray, polynomial_order: int, chunk_sizes: list) -> np.ndarray:
    (m, total_n) = u.shape
    assert sum(chunk_sizes) == total_n, "Sum of chunk sizes must equal number of columns in u"

    theta = np.ones((m, 1))  # Start with constant term
    col_start = 0

    for size in chunk_sizes:
        col_end = col_start + size
        chunk = u[:, col_start:col_end]
        (m_chunk, n_chunk) = chunk.shape

        # Begin with linear terms for this chunk
        chunk_theta = [chunk]

        # Polynomial terms of higher orders
        if polynomial_order >= 2:
            for i in range(n_chunk):
                for j in range(i, n_chunk):
                    chunk_theta.append(chunk[:, i:i+1] * chunk[:, j:j+1])

        if polynomial_order >= 3:
            for i in range(n_chunk):
                for j in range(i, n_chunk):
                    for k in range(j, n_chunk):
                        chunk_theta.append(chunk[:, i:i+1] * chunk[:, j:j+1] * chunk[:, k:k+1])

        # Stack all terms for this chunk
        chunk_terms = np.hstack(chunk_theta)
        theta = np.hstack((theta, chunk_terms))
        col_start = col_end

    return theta

In [11]:
def create_chunked_library_with_transforms(
    u: np.ndarray,
    polynomial_order: int,
    chunk_sizes: list,
    chunk_transforms: list = None  # list of lists like [['sin', 'exp'], [], ['tan']] to apply multiple transforms to each chunk
) -> np.ndarray:
    (m, total_n) = u.shape
    assert sum(chunk_sizes) == total_n, "Sum of chunk sizes must equal number of columns in u"
    if chunk_transforms is None:
        chunk_transforms = [[] for _ in chunk_sizes]
    assert len(chunk_transforms) == len(chunk_sizes), "chunk_transforms must match chunk_sizes length"

    theta = np.ones((m, 1))  # Constant term
    col_start = 0

    for idx, size in enumerate(chunk_sizes):
        col_end = col_start + size
        chunk = u[:, col_start:col_end]
        (m_chunk, n_chunk) = chunk.shape

        chunk_terms = [chunk]  # Linear terms

        # Polynomial terms
        if polynomial_order >= 2:
            for i in range(n_chunk):
                for j in range(i, n_chunk):
                    chunk_terms.append(chunk[:, i:i+1] * chunk[:, j:j+1])

        if polynomial_order >= 3:
            for i in range(n_chunk):
                for j in range(i, n_chunk):
                    for k in range(j, n_chunk):
                        chunk_terms.append(
                            chunk[:, i:i+1] * chunk[:, j:j+1] * chunk[:, k:k+1]
                        )

        # Chunk-specific nonlinear transforms
        transforms = chunk_transforms[idx]
        for transform in transforms:
            if transform == 'sin':
                chunk_terms.append(np.sin(chunk))
            elif transform == 'cos':
                chunk_terms.append(np.cos(chunk))
            elif transform == 'tan':
                chunk_terms.append(np.tan(chunk))
            elif transform == 'exp':
                safe_chunk = np.clip(chunk, a_min=None, a_max=100)
                chunk_terms.append(np.exp(safe_chunk))
            elif transform == 'none':
                chunk_terms.append(chunk)
            else:
                raise ValueError(f"Unknown transform '{transform}' for chunk {idx}")

        chunk_library = np.hstack(chunk_terms)
        theta = np.hstack((theta, chunk_library))
        col_start = col_end

    return theta


In [12]:
# theta = create_basic_library(X, polynomial_order=2)
# theta = create_chunked_library(X, polynomial_order=2, chunk_sizes=[16,  16, 14])
theta = create_chunked_library_with_transforms(
    X, 
    polynomial_order=2,
    chunk_sizes=[16, 16, 14],
    chunk_transforms=[['none'], ['none'], ['exp']]
)

theta.shape

(1164, 470)

In [13]:
def solve_with_lasso(X, y, scale=False, **kwargs):
    # y = y.reshape(-1, 1)

    if scale:
        scaler = StandardScaler(with_mean=False)
        X = scaler.fit_transform(X)

    # Do LASSO regression
    kwargs.setdefault('alpha', 0.05)
    kwargs.setdefault('fit_intercept', False)
    kwargs.setdefault('precompute', True)
    kwargs.setdefault('max_iter', 500000)
    model = Lasso(**kwargs)
    model.fit(X, y)

    # Extract solved coef matrix and remove any scaling effects
    xi = model
    
    if scale:
        xi = scaler.transform(xi)

    return xi

In [14]:
start_dx = time.time()
xi_dx = solve_with_lasso(theta, y_dx)
end_dx = time.time()
print(f"solve_with_lasso for y_dx took {end_dx - start_dx:.4f} seconds")

start_dy = time.time()
xi_dy = solve_with_lasso(theta, y_dy)
end_dy = time.time()
print(f"solve_with_lasso for y_dy took {end_dy - start_dy:.4f} seconds")

C:\Users\lenovo\Desktop\naimish\TC-SINDy-NIO\env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25.476736549640563, tolerance: 0.013204999999999993
  model = cd_fast.enet_coordinate_descent_gram(


solve_with_lasso for y_dx took 22.2049 seconds
solve_with_lasso for y_dy took 22.7015 seconds


C:\Users\lenovo\Desktop\naimish\TC-SINDy-NIO\env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.660760839683068, tolerance: 0.007631000000000001
  model = cd_fast.enet_coordinate_descent_gram(


In [15]:
nonzero_indices_dx = np.where(np.abs(xi_dx.coef_) > 0)[0]
nonzero_indices_dy = np.where(np.abs(xi_dy.coef_) > 0)[0]

print("Nonzero terms for dx:", nonzero_indices_dx.shape)
print("Nonzero terms for dy:", nonzero_indices_dy.shape)

Nonzero terms for dx: (170,)
Nonzero terms for dy: (177,)


In [16]:
L1_value = sum(abs(xi_dx.coef_))
print(L1_value)

0.05315505999845962


In [18]:
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error as rmse

dx_pred = xi_dx.predict(theta)
dy_pred = xi_dy.predict(theta)

r2 = r2_score(y_dx, dx_pred)
print("R² Score (dx):", r2)
r2 = r2_score(dy_pred, y_dy)
print("R² Score (dy):", r2)

mse = rmse(y_dx, dx_pred)
print("RMSE Score (dx):", mse)
mse = rmse(y_dy, dy_pred)
print("RMSE Score (dy):", mse)

R² Score (dx): 0.5828302512292146
R² Score (dy): -0.8082172998038366
RMSE Score (dx): 0.19610954954816343
RMSE Score (dy): 0.1763367143383159
